In [0]:
# Kullanılan kütüphaneler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# Google Drive  ortamına bağlanma
from google.colab import drive
drive.mount('/content/drive')

# Sınıflandırma
Neslihan Oflaz ve Duygu Can


---



>> Makine öğrenmesi ve istatistik alanlarında *sınıflandırma problemi*, temelde yeni bir gözlemin hangi kategoriye ait olduğu sorusuyla ilgilenir. Bir sınıflandırma problemini çözerken elimizdeki gözlemlerden ve bilinen kategorilerinden oluşan bir çalıştırma seti (training set) kullanırız.

Biz bu çalışmada öncelikle ikili sınıflandırmaya yoğunlaşacağız (Birden fazla sınıftan/kategoriden oluşan  problemler  **çoklu sınıflandırma** diye adlandırılır).

**Örnekler:**

+ E-posta: İstenmeyen/İstenen?
+ Online İşlemler: Sahte/Sahte Değil?
+ Tümör: Kötü Huylu/İyi Huylu?

Sınıflandırma problemi bir regresyon problemi olarak ele alabiliriz, yani elimizdeki değişkenlere bağlı olarak yeni bir değişken tahmin etmeye çalışabiliriz. İkili sınıflandırmalarda tahmin etmeye çalıştığımız değişkene *y* dersek:

$$ y \in \{0, 1\} $$
$$ 0: \mbox{Negatif Sınıf}$$
$$ 1: \mbox{Pozitif Sınıf}$$

şeklinde iki değer kullanarak sınıflarımızı belirtebiliriz. Bir regresyon problemi olsa da  sınıflandırma problemlerinde kullanacağımız yöntem doğrusal regresyon   değildir çünkü  girdi ve tahmin arasında aradığımız ilişki doğrusal bir ilişki değildir. Ayrıca doğrusal regresyon sonucu sürekli olacağı için tahminlerimiz kesikli olay uzayımızın dışına düşebilir. Sınflandırma problemini çözmek için uygun olan regresyon algoritması burada bahsedeceğimiz **Lojistik Regresyon**'dur.

## İkili Sınıflandırma Örneği

Bu çalışmada örnek olarak kullanacağımız  veride bir üniversitenin master programına başvuran öğrencilerin iki sınavdan (ALES, TOEFL ya da LES gibi) aldıkları puanlar ve programa kabul edilip edilmedikleri bilgisi bulunuyor.

In [0]:
# Örnek 1 için eğitme verisinin okunması

veri1 = pd.read_csv("/content/drive/My Drive/Makine_Öğrenmesi_Günü/data/log_reg_ex1.csv", names=['sinav_1','sinav_2', 'kabul'])
veri1.head(4)

In [0]:
#@title Örnek-1 : Kabul/Red Edilen Öğrencilerin Sınav Notu Dağılımları
# Verinin Görselleştirilmesi
fig, ax = plt.subplots(figsize=(7,6))

x_kabul = veri1["sinav_1"].loc[veri1['kabul']==1]
y_kabul = veri1["sinav_2"].loc[veri1['kabul']==1] 
x_red = veri1["sinav_1"].loc[veri1['kabul']==0] 
y_red = veri1["sinav_2"].loc[veri1['kabul']==0] 

ax.scatter(x_kabul , y_kabul, marker= r'+', color='Green', label='Kabul')
ax.scatter(x_red , y_red,  marker= r'o',color='Red', label='Red')
ax.set_xlabel('Sınav 1')
ax.set_ylabel('Sınav 2')
ax.legend();
#ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)


Amacımız bu öğrenciler arasında Sınav 1 ve Sınav 2'den  aldıkları  notlara göre kabul alan ve reddedilen öğrenciler olmak üzere iki sınıfa ayırmak. Dahası herhangi bir öğrencinin aldığı sınav notlarına bakarak kabul edilme olasılığını hesaplayabilmek.

Öncelikle kabul edilen ve reddedilen öğrenciler arasında doğrusal bir sınır çizmeye çalışalım, bu sınıra karar sınırı diyeceğiz. Şu anda incelediğimiz örnekte sadece iki nitelik olduğundan (bu örnekte sınavlar) grafikten yararlanarak elimizle de çizebileceğimiz bu sınırı daha sonra Lojistik Regresyon algoritması sayesinde herhangi bir nitelik sayısı için de belirleyebiliriz.

In [0]:
#@title 
# 
fig, ax = plt.subplots(figsize=(7,6))

x_kabul = veri1["sinav_1"].loc[veri1['kabul']==1]
y_kabul = veri1["sinav_2"].loc[veri1['kabul']==1] 
x_red = veri1["sinav_1"].loc[veri1['kabul']==0] 
y_red = veri1["sinav_2"].loc[veri1['kabul']==0] 

ax.scatter(x_kabul , y_kabul, marker= r'+', color='Green', label='Kabul')
ax.scatter(x_red , y_red,  marker= r'o',color='Red', label='Red')

x= np.arange(0,100,0.1)
y= (24 - 0.2*x )/0.2 
ax.plot(x,y)

ax.set_xlabel('Sınav 1')
ax.set_ylabel('Sınav 2')
ax.legend()
ax.set_xlim([25, 100])
ax.set_ylim([25, 100])
ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False);

# Karar Sınırı  

> İki sınıflı bir istatistiksel sınıflandırma probleminde, bir karar sınırı veya karar yüzeyi, temel vektör uzayını her bir sınıf için bir tane olmak üzere iki kümeye ayıran bir (hiper) yüzeydir.

Daha basit bir anlatımla *Karar Sınırı*, farklı sınıfları birbirinden ayıran sınırdır. Elimizde iki nitelik olduğunda karar sınırımızı bir doğru denklemi şeklinde ifade edebiliriz:

$$ \theta^T x =  \theta_0+\theta_1 x_1  + \theta_2 x_2 = 0 $$

Yani karar sınırının grafiği

$$
\theta^T = \left[\begin{array}{ccc}
\theta_0 \\
\theta_1 \\
\theta_2 
\end{array} \right]
$$

vektorünün bileşenleri tarafından belirlenir ve amacımız lojistik regresyon lgoritması kullanarak  bu parametreleri belirlemektir.

Yukarıdaki örnekte  çizdiğimiz karar sınırı $  -24+0.2 x_1  + 0.2 x_2 = 0 $ şeklinde yazılabilir. Bu doğrunun üzerinde kalan alanda bulunan veri noktaları kabul alan öğrencileri belirtirken (y=1), bu doğrunun altında kalanlar ise kabul almayan öğrencileri (y=0) belirtir.

Daha genel ifade edecek olursak

$$ \theta^T x \geq 0 \implies y=1 $$
$$ \theta^T x \lt 0 \implies y=0 $$

olur.

# Hipotez Gösterimi

Doğrusal regresyonda olduğu gibi lojistik regresyonda da bir hipotezden yararlanırız. Bu bölümde hipotezimizi göstermek için kullanacağımız fonksiyonu çalışacağız. Hipotezimizin bize öğrencilerin kabul edilme olasılığını vermesini, bu yüzden de $h_\Theta (x)$ şeklinde gösterilen hipotezimizin 0 ve 1 değerleri arasında kalmasını isteriz:

$$ 0 \leq h_\Theta(x) \leq 1 $$

Bu şartları sağlayan seçimlerden en çok kullanılanı $h_\Theta (x) = g(\Theta^{T}x)$'tir. g fonksiyonu sigmoid ya da lojistik fonksiyon olarak adlandırılır ve 

$$sigmoid(z)= \frac{1}{1+e^{-z}}$$

şeklinde verilir.

## Sigmoid Fonksiyonu

In [0]:
# Sigmoid Fonksiyonu
def sigmoid(z):
  x = np.array(z)            # girdide tutarlılık için
  return 1/(1+np.exp(-x))

In [0]:
#@title Sigmoid Fonksiyonunun Grafiği

fig, ax = plt.subplots(figsize=(7,6))

x= np.arange(-8,8,0.1)
y= sigmoid(x)

ax.plot(x, y)
plt.title('Sigmoid Fonksiyonu')
ax.grid(True)

Daha önce de belirttiğimiz gibi $ h_\theta(x) $, ise çıktımızın 1 olma ihtimalini verir.

$$ h_\theta(x) = \frac{1}{1+e^{-\theta^T x}}$$

Örneğin hipotez çıktısı $h_\theta (x)=0.7 $ bize model çıktımızın (tahmin) %70 ihtimalle 1 olduğunu söyler. Bu bir ikili sınıflandırma problemi olduğu için y'nin 0 ya da 1 olması gerekir ve olasılıklar toplamı 1'e eşit olmalıdır. Aynı hipotez çıktısı tahminimizin %30 ihtimalle 0 olduğunu söyler.

$$ h_\theta(x)=P(y=1|x;\theta)=1−P(y=0|x;\theta) $$

$$ P(y=0|x;θ)+P(y=1|x;\theta)=1 $$

Örnek okul problemimizde kesikli bir sınıflandırma yapabilmek için hipotez fonksiyonu çıktılarımızı aşağıdaki gibi çevirmemiz gerekir:

$$ h_\theta(x)\geq 0.5 \implies y=1 \\
h_\theta(x)\lt 0.5 \implies y=0 $$

Dikkat ederseniz $h_\theta(x) \geq 0.5 $ iken y'yi 1 olarak tahmin ediyoruz. Bu da ancak sigmoid fonksiyonunun girdisinin pozitif olduğu durumlarda doğrudur. Bu da bizi başta bahsettiğimiz geometrik sınıflandırmaya götürür.

$$ \theta^T x \geq 0 \implies y=1 $$
$$ \theta^T x \lt 0 \implies y=0 $$

In [0]:
#@title Karar sınırına yakınlık ve olasılık ilişkisi
# Verinin Görselleştirilmesi
#fig, (ax1, ax2) = plt.subplots(figsize=(7,6))
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(17,7))
#fig.suptitle('Horizontally stacked subplots')

x_kabul = veri1["sinav_1"].loc[veri1['kabul']==1]
y_kabul = veri1["sinav_2"].loc[veri1['kabul']==1] 
x_red = veri1["sinav_1"].loc[veri1['kabul']==0] 
y_red = veri1["sinav_2"].loc[veri1['kabul']==0] 

ax1.scatter(x_kabul , y_kabul, marker= r'+', color='Green', label='Kabul')
ax1.scatter(x_red , y_red,  marker= r'o',color='Red', label='Red')

ax1.set_title("Karar Sınırı")
ax1.set_xlabel('Sınav 1')
ax1.set_ylabel('Sınav 2')
ax1.set_xlim([25, 100])
ax1.set_ylim([25, 100])
ax1.grid(True)

ax1.legend()
#ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)   
x= np.arange(0,100,0.1)
y= (24 - 0.2*x )/0.2 
ax1.plot(x,y, color="Black")
# specific points
ax1.scatter(90,90,marker= r's', color='Cyan',s=50)
ax1.scatter(60,70,marker= r'*', color='fuchsia', s=100)
ax1.scatter(40,40,marker= r'v', color='Orange', s=70)




xs= np.arange(-15,15,0.1)
ys= sigmoid(xs)
theta_ornek = np.array([-24, 0.2, 0.2])
kare = np.dot(theta_ornek,np.array([1, 90, 90]))
yildiz = np.dot(theta_ornek,np.array([1, 60, 70]))
ucgen = np.dot(theta_ornek,np.array([1, 40, 40]))

ax2.plot(xs, ys)
#specific points
ax2.scatter(kare,sigmoid(kare),marker= r's', color='Cyan',s=50)
ax2.scatter(ucgen,sigmoid(ucgen),marker= r'v', color='Orange', s=70)
ax2.scatter(yildiz,sigmoid(yildiz),marker= r'*', color='fuchsia', s=100)
#ax2.set_xlim([-12, 15])
#ax2.set_ylim([-0.1, 1.1])
ax2.set_title('Sigmoid')
ax2.grid(True)


Yukarıda sol taraftaki grafikte sarı üçgenle temsil edilen nokta sınav notları 40,40 olan öğrenciyi, eflatun yıldız sırasıyla 60, 70 olan öğrenciyi,
turkuaz kare ise notları 90, 90 olan öğrenciyi temsil eder. Bu sol taraftaki grafikte işaretlendiği gibi bu öğrencilerin kabul olasılıkları
sırasıyla 0, 0.9 ve 1 olarak hesaplanır. 

## Doğrusal Olmayan Karar Sınırı

Her sınıflandırma probleminde sınıflar doğrusal bir karar sınırı ile ayrılmak zorunda değildir. Örneğin mikroçip üreten bir araştırma şirketinin kalite kontrol testlerini ele alalım. Elimizde iki teste sokulan bu çiplerin kalite kontrolu geçip geçmediklerine dair bilgiler bulunmaktadır.

In [0]:
# Teste tabi tutulan chiplerin test sonuçları

veri2 = pd.read_csv("/content/drive/My Drive/Makine_Öğrenmesi_Günü/data/log_reg_ex2.csv", names=['test_1','test_2', 'onay'])
veri2.head()

Bu verileri çizdirdiğimizde testten kalan ve geçen çiplerin doğrusal bir sınırla rahatlıkla ayrılamacağı görülmektedir. Peki, böyle bir veri kümesi verildiğinde lojistik regresyonu veriye nasıl uydurmalıyız?

In [0]:
#@title Örnek-2: Kaliteli ve Kalitesiz Mikroçiplerin  Dağılımı
# Decision Boundary
fig, ax = plt.subplots(figsize=(8,8))

x_iyi= veri2["test_1"].loc[veri2['onay']==1]
y_iyi = veri2["test_2"].loc[veri2['onay']==1] 
x_kotu = veri2["test_1"].loc[veri2['onay']==0] 
y_kotu = veri2["test_2"].loc[veri2['onay']==0] 
theta = np.linspace(0, 2 * np.pi, 50)

ax.scatter(x_iyi , y_iyi, marker= r'+', color='Green', label='Kaliteli')
ax.scatter(x_kotu , y_kotu,  marker= r'o',color='Red', label='Kalitesiz')

ax.set_xlabel('Mikroçip Testi 1')
ax.set_ylabel('Mikroçip Testi 2')
ax.legend();

Polinom regresyonunda, doğrusal olmayan uyumlama yapabilmek fazladan yüksek dereceli nitelikler eklemiştik. Benzer şekilde, lojistik regresyonla da eğri karar sınırları yakalayabilmek için böyle terimler ekleyeceğiz. Sonuçta sigmoid fonksiyonunun girdisinin doğrusal olmadığı durumlarda çıktısı da doğrusal değildir. Örneğin:

$$ \theta^T x =  \theta_0+\theta_1 x_1^2 + \theta_2 x_2^2 + \theta_3 x_1x_2$$

Yukarıdaki denklemi 0'a eşitlediğimizde doğrusal olmayan bir *Karar Sınırı* elde ederiz.

In [0]:
#@title Doğrusal Olmayan Karar Sınırı Örneği
# Decision Boundary
fig, ax = plt.subplots(figsize=(8,8))

x_iyi= veri2["test_1"].loc[veri2['onay']==1]
y_iyi = veri2["test_2"].loc[veri2['onay']==1] 
x_kotu = veri2["test_1"].loc[veri2['onay']==0] 
y_kotu = veri2["test_2"].loc[veri2['onay']==0] 
theta = np.linspace(0, 2 * np.pi, 50)
r= 1/2
x = (r+0.27)* np.sin(theta-np.pi/6) + 0.10
y = (r+0.12) * np.cos(theta) + 0.15

plt.plot(x,y)
ax.scatter(x_iyi , y_iyi, marker= r'+', color='Green', label='Kaliteli')
ax.scatter(x_kotu , y_kotu,  marker= r'o',color='Red', label='Kalitesiz')

ax.set_xlabel('Kalite Testi 1')
ax.set_ylabel('Kalite Testi 2')
ax.legend()
a=0

# Bedel Fonksiyonu

Bu bölümde lojistik regresyonun parametrelerini veri setine uydurmak için gereken optimizasyon hedefini tanımlayacağız. Şimdiye kadar  *en iyi* karar sınırını ve *doğru* hipotezi veren parametreleri ($\mathbf\theta$ vektörünü oluşturan parametreleri), yani hep *en iyi* karar sınırını ve *doğru* hipotezimizi bildiğimizi varsayarak birtakım çıkarımlarda bulunduk. Gerçekten de örnekte kullandığımız parametreler bize iyi bir sınıflandırma sağlar. Fakat özellikle ikiden fazla niteliğin varlığında iyi bir karar sınırı verecek  parametreleri elle hesaplayarak belirlemek pek mümkün değildir.


**Peki bu parametreleri ($\theta$'ları) nasıl belirleriz?**
+ *En hatasız* hipotezi verecek parametreleri seçerek. Bunu yaparken Doğrusal Regresyonda olduğu gibi bir bedel fonksiyonu tanımlayacağız. 


m adet örnek ve n adet nitelik içeren bir eğitim verisi(traning data) alalım:

$$ \mbox{Eğitim Verisi:} {(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), ..., (x^{(m)}, y^{(m)}) } $$

$$ \begin{equation*}
x \in
\begin{bmatrix}
x_{0} \\
x_{1} \\
\vdots \\
x_{n} 
\end{bmatrix}
\end{equation*}$$

$$ h_\theta(x) = \frac{1}{1+e^{-\theta^T x}}$$

In [0]:
# Hipotez
def hipotez(X, theta):
 return sigmoid(np.dot(X,theta)) 

In [0]:
#@title Örnek-1 
veri1.head(7)

In [0]:
veri1.shape

Örnek-1 için 2 nitelik vardır (sınav1 ve sınav2),  yani n=2'dir. Eğitim datasının sadece ilk 6 satırını göstersek de 100 örnek satırı (100 öğrenci) vardır, bu yüzden m=100'dür.

In [0]:
# Örnek-1 

X0 = np.array(veri1[['sinav_1', 'sinav_2']])   # nitelikler
y1 = np.array(veri1['kabul'])                  # gerçek değer vektörü
m = len(y1)                                    
birler = np.ones(m)                            
X1 = np.column_stack((birler, X0))             # birler sutununun eklenmesi

m adet örneğe n adet niteliğe sahip eğitim verisinde i. satır için $x_0^{(i)} =1$ olmak üzere:

$$ \begin{equation*}
x^{(i)} =
\begin{bmatrix}
x^{(i)}_{0} \\
x^{(i)}_{1} \\
\vdots \\
x^{(i)}_{n} 
\end{bmatrix},  \quad   y^{(i)}= 0 \mbox{ veya } 1 \mbox{'dir. }
\end{equation*}$$

$$\theta^T\cdot x^{(i)} = \sum_{j=0}^n \theta_j x^{(i)}_j$$

olmak üzere hipotezimiz 
$$ h_\theta(x^{(i)}) = \frac{1}{1+e^{-\theta^T x^{(i)}}}$$ 
her satır için olasılık/tahmin verir.

Ve her bir tahminin  gerçekten ne kadar saptığını $bedel(h_\theta(x^{(i)}), y^{(i)})$ ile ölçersek toplam bedel

$$ J(\theta) = \sum_{i=1}^m bedel(h_\theta(x^{(i)}), y^{(i)}) $$

olarak hesaplanır.

Bedel fonksiyonunu Doğrusal Regresyon'un bedel fonksiyonunda olduğu gibi tahminin gerçek sınıftan sapmasının karesi olarak tanımlarsak konveks olmayan bir bedel fonksiyonu elde ederiz ve Dereceli Alçalma algoritmasının bu bedel fonksiyonunun global minimumunu bulması garanti değildir (lokal minimalara takılabilir). Bu yüzden *Lojistik Regresyon* için bedel fonksiyonu aşağıdaki gibi tanımlanır:


$$\begin{equation*}\\
bedel(h_\theta(x), y) = \left\{
\begin{array}{rl}
-log(h_\theta(x)) & \text{eğer } y = 1,\\
-log(1 -h_\theta(x)) & \text{eğer } y = 0.
\end{array} \right.
\end{equation*}
$$ 



Gerçekte $y = 1$ iken, model tahmini $h_\theta$ bire yaklaştıkça *Bedel* azalır; aksi yönde artar!

Benzer şekilde $y = 0$ iken, model tahmini $h_\theta$ sıfıra yaklaştıkça *Bedel* azalır; aksi yönde artar!

In [0]:
#@title J'nin hipoteze göre değişimi
# Bedel fonksiyonunun y-1 ve y=0 durumundaki grafikleri
fig, (ax1, ax2)= plt.subplots(1,2,figsize=(14,6))
def J1(x):
  #eps = 10^-3
  return -np.log(x)
def J0(x):
  #eps = 10^-3
  return -np.log(1-x)
x = np.arange(0.0001,1,0.01)
y = J1(x)
ax1.plot(x,J1(x), "-")
ax1.set_title("y=1 durumunda:")
ax2.plot(x,J0(x), "-")
ax2.set_title("y=0 durumunda:")
a=0

## Basitleştirilmiş Bedel Fonksiyonu

**y = 1** ve **y = 0** durumları için ayrı ayrı yazılan *Bedel Fonksiyon*'unu tek bir formülde gösterebiliriz. 

$$ \mathrm{bedel}(h_\theta(x),y) = - y \; \log(h_\theta(x)) - (1 - y) \log(1 - h_\theta(x)) $$

**y = 1** iken ikinci terim, **y=0** iken ilk terim yok olacaktır ve *Bedel Fonksiyonu* eski halini alacaktır. Sadece bu daha *sıkıştırılmış* bir gösterimdir.

Konveks şekilli bu bedel fonksiyonu ayrıca istatistikten bildiğimiz En Yüksek Olabilirlik Kestirimi (Maximum Likelihood Estimation) ile elde edilebilir.

Tüm *Bedel Fonksiyonu* ise aşağıdaki gibi ifade edilir:

$$ J(\theta) = -\frac{1}{m}\sum_{i=1}^m [y^{(i)} \; \log(h_\theta(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)}))] $$

In [0]:
# Bedel fonksiyonu

def bedel(theta, X, y):     # X1: bir sutunu eklenmiş nitelik matrisi; y: gerçek değer vektörü; theta: parametre vektörü;
  m = len(y)      
  h = hipotez(X, theta)  
  eps = 0.0001              #log(0) hatasini onlemek icin

  J = 1/m*(np.dot(y, -np.log(h+eps)) + np.dot(1-y,-np.log(1-h+eps)))

  return J

Bu bedel fonksiyonu verilmişken, iki sınıfı birbirinden en iyi ayıran karar sınırını bulmak için fonksiyonu $\theta$'ya göre minimize etmek gerekir. Böylelikle en iyi parametre seti bize en düşük bedeli verecektir. 

$$ \underset{\theta}{\text{min}} J(\theta)  $$

En düşük bedeli veren parametre seti hipotez fonksiyonuna konulunca, yeni bir örnek, *x* için model tahminimizi bulabiliriz.
$$ h_\theta(x) = \frac{1}{1+e^{-\theta^T x}}$$

Mesela yüksek lisansa kabul alma örneğinde,
$$ \theta = [-24\ \  0.2\ \ 0.2\ ]$$ 
için bedeli hesaplayalım.

In [0]:
# TEST bedel fonksiyonu

theta_ornek = np.array([-24, 0.2, 0.2])
bedel_ornek = bedel(theta_ornek, X1, y1)
print('bedel=', bedel_ornek.round(3))
#passed

Düşük bir bedel değerine sahip hipotezimiz (modelimiz), iki sınıfı birbirinden iyi ayırmaktadır.

In [0]:
#@title bedel = 0.218
# 
fig, ax = plt.subplots(figsize=(5,4))

x_kabul = veri1["sinav_1"].loc[veri1['kabul']==1]
y_kabul = veri1["sinav_2"].loc[veri1['kabul']==1] 
x_red = veri1["sinav_1"].loc[veri1['kabul']==0] 
y_red = veri1["sinav_2"].loc[veri1['kabul']==0] 
x= np.arange(0,100,0.1)
y= (24 - 0.2*x )/0.2 
ax.scatter(x_kabul , y_kabul, marker= r'+', color='Green', label='Kabul')
ax.scatter(x_red , y_red,  marker= r'o',color='Red', label='Red')

ax.plot(x,y)
ax.set_xlabel('Sınav 1')
ax.set_ylabel('Sınav 2')
ax.legend()
ax.set_xlim([30, 100])
ax.set_ylim([30, 100])
ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)

# Dereceli Alçalma

Bedeli minimize etmek için *Dereceli Alçalma* algoritmasını kullanacağız. Dereceli Alçalmanın genel algoritmasının aşağıdaki gibi olduğunu hatırlayalım.
\begin{align*}& Tekrar \; \lbrace \newline & \; \theta_j := \theta_j - \alpha \dfrac{\partial}{\partial \theta_j}J(\theta) \newline & \rbrace
\end{align*}
Bu algoritmanın tüm parametreleri ($\theta$) her adımda eş zamanlı güncellediğini unutmamak gerekir.
Lojistik Regresyon bedel fonksiyonunu yerine koyup tekrar yazarsak:

\begin{align*} & Tekrar \; \lbrace \newline & \; \theta_j := \theta_j - \frac{\alpha}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \newline & \rbrace \end{align*}

eşitliğini elde ederiz. Bu denklem *Doğrusal Regresyon* parametre güncelleme kuralı ile aynı görünüşte olsa da $h_\theta(x^{(i)})$ sigmoid formunda olacağı için aslında farklıdır.

*Dereceli Alçalma* algoritmasının vektörize implementasyonu ($\theta:= \theta− \frac{\alpha}{m}X^T (g(Xθ)− \vec{y})$) aşağıdaki  gibidir. Fakat pratikte çok yavaş çalıştığı için bedeli minimize etmek için bir optimizasyon paketinden yararlanacağız.

In [0]:
# Derece fonksiyonu 

def derece(theta, X, y):
  m = len(y)
  h = hipotez(X,theta)
  
  # bedel derecesi
  grad = 1/m*np.dot(np.transpose(X), (h-y))
  
  return grad

In [0]:
derece(theta_ornek, X1, y1)

In [0]:
# Dereceli alçalma

def dereceli_alcalt(it, alpha,  X, y):  # alpha: öğrenme parametresi
  ## egitilmemis theta
  m , n = X.shape
  theta0 = np.zeros(n)

  for i in range(0,it):                 # it: iterasyon sayısı
    grad = derece(theta0, X, y)
    theta0 = theta0 - alpha*grad

  return theta0

 Yavaş çalışmaması için dereceli alçalma fonksiyonunu az iterasyonla kullanırsak bedel fonksiyonu için pek de iyi bir sonuç alamadığımızı görüyoruz. Bu fonksiyonun doğru global minimuma yakınsamadan iterasyonun bitmesinden kaynaklanmaktadır.

In [0]:
# 10000 iterasyon
theta_deneme = dereceli_alcalt(10000, 0.3, X1, y1)
bedel(theta_deneme, X1, y1)

İterasyon sayısını artırdıkça cevap iyileşecektir. 

In [0]:
# 100000 iterasyon
theta_deneme = dereceli_alcalt(100000, 0.3, X1, y1)
bedel(theta_deneme, X1, y1)

Nitelikler aynı aralığa ölçeklendirilirse *Dereceli Alçalma* algoritması sonuca daha hızlı yakınsayacaktır.

### İLERİ OPTİMİZASYON TEKNİKLERİ

*Eşlenik Gradyan*, *BFGS*, *L-BFGS* gibi karmaşık algoritmalar her iterasyonda uygun öğrenme hızını seçerek sonuca *Dereceli Azalım* algoritmasından daha hızlı yakınsarlar.

Örneğin *SciPy* kütüphanesindeki *fmin_tnc* fonksiyonu *Newton Eşlenik Gradyan* algoritmasının implementasyonudur.

In [0]:
# Bedel fonksiyonunun fmin_tnc ile minimize edilmesi

import scipy.optimize as op
theta0 = np.zeros(3)  

optim=  op.fmin_tnc(func=bedel, x0=theta0, args=(X1, y1),approx_grad=True)

theta_op = optim[0]


Hazır bir optimizayon kütüphanesinden kullandığımız fonksiyonla çok daha iyi bir sonuç elde ediyoruz:

In [0]:
bedel(theta_op, X1, y1)

In [0]:
# fmin_tnc ile buldugumuz theta vektoru
theta_op.round(3)

In [0]:
#@title bedel = 0.203
# 
fig, ax = plt.subplots(figsize=(8,6))

x_kabul = veri1["sinav_1"].loc[veri1['kabul']==1]
y_kabul = veri1["sinav_2"].loc[veri1['kabul']==1] 
x_red = veri1["sinav_1"].loc[veri1['kabul']==0] 
y_red = veri1["sinav_2"].loc[veri1['kabul']==0] 
x= np.arange(0,100,0.1)
y= (-theta_op[0] -theta_op[1] *x )/theta_op[2]
ax.scatter(x_kabul , y_kabul, marker= r'+', color='Green', label='Kabul')
ax.scatter(x_red , y_red,  marker= r'o',color='Red', label='Red')

ax.plot(x,y)
ax.set_xlabel('Sınav 1')
ax.set_ylabel('Sınav 2')
ax.legend()
ax.set_xlim([30, 100])
ax.set_ylim([30, 100])
ax.tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)

## Çok Sınıflı Sınıflandırma

Şimdi ikiden fazla kategoriye sahip olduğumuzda verinin sınıflandırmasına değineceğiz. Mesela gelen e-postaları iş, aile, arkadaşlar ya da hobi olarak otomatik tasnif eden bir makine öğrenmesi modeli bir çoklu sınıflandırma problemidir.
$Y = {0,1}$ yerine tanımımızı genişleteceğiz, $y = {0,1 ... n}$. Y = {0,1 ... n} olduğundan, sorunumuzu n + 1'e (+1 olarak bölüyoruz, çünkü dizin 0'da başlıyor) ikili sınıflandırma problemleri; her birinde, 'y'nin sınıflarımızdan birinin üyesi olma ihtimalini tahmin ediyoruz.

\begin{align*}& y \in \lbrace0, 1 ... n\rbrace \newline& h_\theta^{(0)}(x) = P(y = 0 | x ; \theta) \newline& h_\theta^{(1)}(x) = P(y = 1 | x ; \theta) \newline& \cdots \newline& h_\theta^{(n)}(x) = P(y = n | x ; \theta) \newline& \mathrm{prediction} = \max_i( h_\theta ^{(i)}(x) )\newline\end{align*}

## Birine Karşıt Hepsi (One-vs-All)

Temel olarak bir sınıfı seçiyoruz ve ardından diğerlerini tek bir ikinci sınıfa ayırıyoruz. Bunu her seferinde ikili lojistik regresyon uygulayarak tekrar tekrar yapıyoruz ve sonra en yüksek değeri döndüren hipotezi tahminimiz olarak kullanıyoruz.

Aşağıdaki resim, birinin 3 sınıfı nasıl sınıflandırabileceğini göstermektedir:

![](https://drive.google.com/uc?id=1vW3mXSJWGZnOuwtEi5k9IH4BwpcVse4r)

## Aşırı Öğrenme (Overfitting)

Y'yi x ∈ R'den tahmin etme problemini düşünün. Aşağıdaki en soldaki şekil, bir $y = \theta_0 + \theta_1 x$ veri setine uydurma sonucunu göstermektedir. Verilerin gerçekten düz bir çizgide bulunmadığını görüyoruz ve bu nedenle uygunluk çok iyi değil.
 
![](https://drive.google.com/uc?id=1vykgbJmIkxRUa5qfPmkhdpN6-jfO3_WR)

Ortadaki şekil ikinci dereceden $ y = \theta_0 + \theta_1x + \theta_2x^2$ denkleminin veriye uydurulmasıdır. Ne kadar çok yeni nitelik eklersek eğri noktalara daha iyi uyumsar.  En sağdaki şekil 5. dereceden bir polinomun uyumsamasıdır: $ y = \sum_{j=0} ^5 \theta_j x^j$. Artık eğri her noktadan geçse de bu model iyi bir tahminleyici değildir çünkü veriyi ezberlemiştir. Yeni bir örnek geldiğinde sonucu genelleyemez.
Özetle en soldaki grafik yetersiz-uyum (underfitting), en sağdaki grafik ise aşırı-uyum (overfitting) örneğidir. 

Yetersiz uyum (underfitting) ya da yüksek önyargı (high bias), hipotez fonksiyonumuzun biçiminin veri eğilimi yakalayamasıdır. Genellikle çok basit veya çok az özellik kullanan bir işlevden kaynaklanır. Diğer uçta ise, aşırı-uydurma veya yüksek varyans, mevcut verilere uyan ancak yeni verileri öngörmek için iyi genelleme yapmayan bir hipotez fonksiyonundan kaynaklanır. Genellikle verilerle ilgisi olmayan birçok gereksiz eğri ve açı oluşturan karmaşık bir fonksiyonu model olarak kullanmaktan kaynaklanır.

Bu terminoloji hem doğrusal hem de lojistik regresyona uygulanır. Aşırı uyum (overfitting) sorununu ile başa çıkmak için iki yol vardır:

1. Nitelik sayısını azaltmak:
  + Elle seçerek
  + Bir model seçim algoritmasını kullanarak

2. Düzenlileştirme
  + Tüm nitelikleri koruyarak, ancak parametre $\theta_j$ büyüklüklerini küçülterek
  + Düzenlileştirme, pek çok biraz kullanışlı nitelikliğe sahip olduğumuzda iyi çalışır.

## Düzenlileştirme 

Düzenlileştirme için *Bedel Fonksiyonu*'nun sonuna parametre büyüklüklerini sınırlayıcı bir terim ekleriz.
![](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/Od9mobDaEeaCrQqTpeD5ng_4f5e9c71d1aa285c1152ed4262f019c1_Screenshot-2016-11-22-09.31.21.png?expiry=1572652800000&hmac=MM27fp24SjI7PLK9dY6cRJleWDnHFIbAm-Pxaz1Q_Hs)

*Bedel Fonksiyonu*nun düzenlileştirme altındaki ifadesi şuna dönüşür:

$$ J(\theta) = \frac{-1}{m}\sum_{i=1}^m [y^{(i)} \; \log(h_\theta(x^{(i)})) + (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)}))] + 
\frac{\lambda}{2m}\sum_{j=1}^m \theta_j^2 $$



In [0]:
# Regülarize edilmiş bedel fonksiyonu

def reg_bedel(theta, lmbda, X, y):             # X: nitelik matrisi, y: gerçek değer vektörü, theta: parametre vektörü, lmbda: regülarizasyon parametresi
  m = len(y)
  h = sigmoid(np.dot(X,theta))                 # hipotez
  
  eps = 0.0001                                 # log(0) hatasini onlemek icin
  J0 = bedel(theta, X, y )
  J_lmbda = lmbda/(2*m)*np.dot(theta, theta)   # regularizasyon  terimi
  
  return J0 + J_lmbda

*Bedel Fonksiyonu*'nu $\theta$'ya göre minimize ettiğimizde ($ \underset{\theta}{\text{min }} J(\theta)$), sondaki toplam terimi $\theta$'ları $\lambda$, düzenlileştirme parametresi ile sınırlayacaktır. *Bedel Fonksiyonu*'muzu böyle değiştirdiğimizde *Dereceli Alçalma* algoritmamız da aşağıdaki gibi değişir:



\begin{align*}  Tekrar  \; \lbrace \newline  \;
&\theta_0 := \theta_0 - \frac{\alpha}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)}) x_0^{(i)} \newline 
&\theta_j := \theta_j - \frac{\alpha}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} + \frac{\lambda}{m} \theta_j \newline & \rbrace \end{align*}

Dikkat ederseniz sondaki toplam teriminin indeksi önyargı (bias) terimini atlamak için 1'den başlatılmıştır. Bu yüzden *Dereceli Alçalma* algoritması içinde $\theta_0$ ayrıca güncelleştirilir. 

In [0]:
# Regülarize edilmiş derece fonksiyonu

def reg_derece(theta, X, y):
  m = len(y)
  h = hipotez(X,theta)                          # hipotez
  
  grad = 1/m*np.dot(np.transpose(X), (h-y))     # derece
  grad_lmbda = lmbda/m*theta                    # regularizasyon terimi
  
  return grad+grad_lmbda

![](https://drive.google.com/uc?id=1fIQaDYsd0th2dlAPcl_PLI-iK_XsP41U)

## Kaynakça

+ [İstatistiki Sınıflandırma](https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly90ci53aWtpcGVkaWEub3JnL3dpa2kvxLBzdGF0aXN0aWtpX3PEsW7EsWZsYW5kxLFybWE)
+ [Decision Boundary](https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvRGVjaXNpb25fYm91bmRhcnk)

+ [Deriving cost function using MLE :Why use log function?](https://math.stackexchange.com/questions/886555/deriving-cost-function-using-mle-why-use-log-function)